## 1. Request Data from Star Wars API
This part is modified based on `request.py` from the Udacity Data Engineer Project.

In [1]:
import  requests
import json

In [5]:
class RequestStarWars:
    def __init__(self):
        self._base_urls = {
            "films": "https://swapi.dev/api/films/",
            "people": "https://swapi.dev/api/people/",
            "planets": "https://swapi.dev/api/planets/",
            "species": "https://swapi.dev/api/species/",
            "starships": "https://swapi.dev/api/starships/",
            "vehicles": "https://swapi.dev/api/vehicles/"
        }

    def get_content(self, type):
        url = self._base_urls[type]
        all_data = []

        while url:
            response = requests.get(url)
            if response.status_code == 200:
                response_data = response.json()
                all_data.extend(response_data['results'])
                
                url = response_data["next"]

            else:
                print(f"Request completed with Error. Response Code : {response.status_code}")
                break
        return all_data

In [6]:
import os
os.makedirs('data', exist_ok=True)

request_api = RequestStarWars()

table_names = ['people', 'films', 'starships', 'vehicles', 'species', 'planets']

for table_name in table_names:
    data = request_api.get_content(table_name)
    with open(f'data/{table_name}.json', 'w') as file:
        json.dump(data, file, indent=4)

Request completed with Error. Response Code : 404


In [3]:
# examine the content
with open(f'data/{table_name}.json', 'w') as file:
        json.dump(data, file, indent=4)
['people'][0]['films']

NameError: name 'data_dict' is not defined

## 2. Create Schemas and Build Connections

In [14]:
# define the data schmea and write relevant SQL statements according to the documentation
# the main change here is add id column for each table

create_tables = ['''
CREATE TABLE IF NOT EXISTS people (
    id INTEGER PRIMARY KEY,
    name TEXT,
    birth_year TEXT,
    eye_color TEXT,
    gender TEXT,
    hair_color TEXT,
    height TEXT,
    mass TEXT,
    skin_color TEXT,
    homeworld TEXT,
    films TEXT, 
    species TEXT,
    starships TEXT,
    vehicles TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY,
    title TEXT,
    episode_id INTEGER,
    opening_crawl TEXT,
    director TEXT,
    producer TEXT,
    release_date DATE,
    species TEXT,
    starships TEXT,
    vehicles TEXT,
    characters TEXT, 
    planets TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS starships (
    id INTEGER PRIMARY KEY,
    name TEXT,
    model TEXT,
    starship_class TEXT,
    manufacturer TEXT,
    cost_in_credits TEXT,
    length TEXT,
    crew TEXT,
    passengers TEXT,
    max_atmosphering_speed TEXT,
    hyperdrive_rating TEXT,
    MGLT TEXT,
    cargo_capacity TEXT,
    consumables TEXT,
    films TEXT,
    pilots TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS vehicles (
    id INTEGER PRIMARY KEY,
    name TEXT,
    model TEXT,
    vehicle_class TEXT,
    manufacturer TEXT,
    length TEXT,
    cost_in_credits TEXT,
    crew TEXT,
    passengers TEXT,
    max_atmosphering_speed TEXT,
    cargo_capacity TEXT,
    consumables TEXT,
    films TEXT,
    pilots TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS species (
    id INTEGER PRIMARY KEY,
    name TEXT,
    average_height TEXT,
    average_lifespan TEXT,
    classification TEXT,
    designation TEXT,
    eye_colors TEXT,
    hair_colors TEXT,
    homeworld TEXT,
    language TEXT,
    skin_colors TEXT,
    people TEXT,
    films TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
''',
'''
CREATE TABLE IF NOT EXISTS planets (
    id INTEGER PRIMARY KEY,
    name TEXT,
    diameter TEXT,
    rotation_period TEXT,
    orbital_period TEXT,
    gravity TEXT,
    population TEXT,
    climate TEXT,
    terrain TEXT,
    surface_water TEXT,
    url TEXT,
    created TEXT,
    edited TEXT
);
'''
]

In [20]:
# pick up some connections that i'm interested in
create_connections = [
    '''
    -- people_films
    CREATE TABLE IF NOT EXISTS people_films (
    person_id INTEGER,
    film_id INTEGER,
    PRIMARY KEY (person_id, film_id),
    FOREIGN KEY (person_id) REFERENCES people(id),
    FOREIGN KEY (film_id) REFERENCES films(id));
    ''',
    '''
    --people_species
    CREATE TABLE IF NOT EXISTS people_species (
    person_id INTEGER,
    species_id INTEGER,
    PRIMARY KEY (person_id, species_id),
    FOREIGN KEY (person_id) REFERENCES people(id),
    FOREIGN KEY (species_id) REFERENCES species(id)
    );
    ''',
    ''' 
    --people_starships
    CREATE TABLE IF NOT EXISTS people_starships (
    person_id INTEGER,
    starship_id INTEGER,
    PRIMARY KEY (person_id, starship_id),
    FOREIGN KEY (person_id) REFERENCES people(id),
    FOREIGN KEY (starship_id) REFERENCES starships(id)
    );
    ''',
    '''
    -- people_vehicles
    CREATE TABLE IF NOT EXISTS people_vehicles (
    person_id INTEGER,
    vehicle_id INTEGER,
    PRIMARY KEY (person_id, vehicle_id),
    FOREIGN KEY (person_id) REFERENCES people(id),
    FOREIGN KEY (vehicle_id) REFERENCES vehicles(id)
    );
    '''
]

## 3. Insert values to the tables

In [21]:
# connections are stored in the urls, so we create a function to convert them into ids
def extract_id(url):
    return int(url.strip('/').split('/')[-1])